In [4]:
from scipy import stats
import os
import numpy as np

### Example

In [25]:
# we wish to test the hypothesis H0 > H1, 
# where H0 is the mean returns achieved by D-Shape, 
# and H1 is the mean return achieved by best baseline

alpha = 0.05
# H0: d = 1st test - 2nd test = 0 
# H1: d = 1st test - 2nd test < 0

# first_test =[23, 20, 19, 21, 18, 20, 18, 17, 23, 16, 19]
# second_test=[24, 19, 22, 18, 20, 22, 20, 20, 23, 20, 18]
first_test =[23, 24]
second_test=[24, 25]

# use ttest_rel if measurements come from same underlying population
# ttest_ind if all measurements are independent.
t_value,p_value=stats.ttest_ind(first_test,second_test)
one_tailed_p_value=float("{:.6f}".format(p_value/2)) 

print('Test statistic is %f'%float("{:.6f}".format(t_value)))
print('p-value for one_tailed_test is %f'%one_tailed_p_value)


if one_tailed_p_value<=alpha:
    print('Conclusion','n','Since p-value(=%f)'%one_tailed_p_value,'<','alpha(=%.2f)'%alpha,'''We reject the null hypothesis H0. 
So we conclude that the students have benefited by the tuition class. i.e., d = 0 at %.2f level of significance.'''%alpha)

else:
    print('Conclusion','n','Since p-value(=%f)'%one_tailed_p_value,'>','alpha(=%.2f)'%alpha,'''We do not reject the null hypothesis H0. 
So we conclude that the students have not benefited by the tuition class. i.e., d = 0 at %.2f level of significance.'''%alpha)

Test statistic is -1.414214
p-value for one_tailed_test is 0.146447
Conclusion n Since p-value(=0.146447) > alpha(=0.05) We do not reject the null hypothesis H0. 
So we conclude that the students have not benefited by the tuition class. i.e., d = 0 at 0.05 level of significance.


## DShape

In [33]:
def compile_logs(results_dir, expt_name, n_trials):
    env_rets = []
    for trial_idx in range(n_trials):
        load_path = os.path.join(results_dir, expt_name, f"trial={trial_idx}", "logs.npz")
        res = np.load(load_path, allow_pickle=True)
        env_ret_trial = res["env_ret"] # learning curve for single trial
        if env_ret_trial.shape[0] == 500:
            print(load_path)
        env_rets.append(env_ret_trial)
        
    env_rets = np.array(env_rets)
#     means = np.mean(env_rets, axis=0)
    return env_rets, res["eval_ts"]


def measure_sample_eff(expt_name, results_dir, gridworld_size, n_trials=30, desired_value=None):
    expt_name = f"{expt_name}_world=basic_size={gridworld_size}"
    opt_ret_tss = []
    for trial_idx in range(n_trials):
        log_path = os.path.join(results_dir, expt_name, f"trial={trial_idx}", "logs.npz")
        data = np.load(log_path)
        if desired_value is None:
            desired_value = float(data['opt_rew'])
        idxs = np.where(data['env_ret'] >= desired_value)[0]
        if len(idxs)>0:
            first_idx = min(idxs) 
            opt_ret_tss.append(data['eval_ts'][first_idx]) # ts when optimal return achieved
        else: # opt rew not reached
            print(f"Warning: desired value not reached for expt {expt_name}, trial {trial_idx}. Appending last timestep.")
            opt_ret_tss.append(data['eval_ts'][-1])
    return np.mean(opt_ret_tss), np.std(opt_ret_tss)

def measure_robust_subopt(baseline_expt:str, test_expt_names:list,
                          gridworld_size, results_dir, desired_value=None):
    res_dict = {}
    baseline_sample_eff = measure_sample_eff(baseline_expt, results_dir, gridworld_size,
                                            desired_value=desired_value)[0] 
    res_dict["baseline"] = baseline_sample_eff

    for test_expt in test_expt_names:
        test_sample_eff = measure_sample_eff(test_expt, results_dir, gridworld_size, 
                                             desired_value=desired_value)[0] 
        res_dict[test_expt] = test_sample_eff - baseline_sample_eff
    return res_dict

In [16]:
results_dir = "/scratch/cluster/clw4542/gridworld_results2/"
expt_basename = "dshape" # "manhattan"
world_size = 10

dshape_env_rets, eval_ts = compile_logs(results_dir, 
             expt_name=f"{expt_basename}_world=basic_size={world_size}",
             n_trials=30)
manhattan_env_rets, eval_ts = compile_logs(results_dir, 
             expt_name=f"manhattan_world=basic_size={world_size}",
             n_trials=30)

In [35]:
eval_ts[10] 

50232

In [38]:
# measure average timesteps to convergence
measure_sample_eff("dshape", results_dir, gridworld_size=world_size, n_trials=30, desired_value=None)

(61011.73333333333, 8205.561755839079)

In [39]:
measure_sample_eff("manhattan", results_dir, gridworld_size=world_size, n_trials=30, desired_value=None)

(130949.56666666667, 4836.32444240688)

In [37]:
# we wish to test the hypothesis H0 > H1, 
# where H0 (null hypothesis) is that there is no significant difference between DShape and Baseline, 
# and H1 is that DShape is better than Manhattan
# H0: d = Manhattan - DShape = 0 
# H1: d = Manhattan - DShape < 0

train_log_step = 10
alpha = 0.05

t_value,p_value=stats.ttest_ind(manhattan_env_rets[:, train_log_step],dshape_env_rets[:, train_log_step])
one_tailed_p_value=float("{:.6f}".format(p_value/2)) 
print(f'Test statistic is {t_value}')
print(f'p-value for one_tailed_test is {one_tailed_p_value}, alpha is {alpha}')

Test statistic is -11.273264041980456
p-value for one_tailed_test is 0.0, alpha is 0.05
